In [1]:
require('nn')
require('hdf5')
require('optim')

f = hdf5.open("data.hdf5", "r")

train_rnn_X = f:read('train_rnn_X'):all()
train_rnn_Y = f:read('train_rnn_Y'):all()
train_X_seqeuence =  f:read('train_X_sequence'):all()
train_Y  = f:read('train_Y'):all()
valid_reduced_X = f:read('valid_reduced_X'):all()
valid_reduced_Y = f:read('valid_reduced_Y'):all()
windows_train = f:read('windows_train'):all()
windows_valid = f:read('windows_valid'):all()
test = f:read('test'):all()
valid_kaggle_with_spaces = f:read('valid_kaggle_with_spaces'):all()
valid_kaggle_without_spaces = f:read('valid_kaggle_without_spaces'):all()
valid_answers = f:read('valid_answers'):all()
nfeatures = (f:read('nfeatures'):all())[1]
nclasses = (f:read('nclasses'):all())[1]

f:close()

window_size = windows_train:size(2)
n_actual = nclasses

In [2]:
embedded_size = 15
D_h = 100

lookup1 = nn.LookupTable(nfeatures*window_size,embedded_size)
linear1 = nn.Linear(window_size*embedded_size, D_h)
linear2 = nn.Linear(D_h, n_actual)
h = nn.Sequential()
h:add(lookup1)
--lookup1.weight:renorm(2, 2, 1)
h:add(nn.Reshape(window_size*embedded_size, True))
h:add(linear1)
h:add(nn.Tanh())
h:add(linear2)
h:add(nn.LogSoftMax())
mse = nn.ClassNLLCriterion()
--mse = nn.CrossEntropyCriterion()
w, dl_dw = h:getParameters()

In [3]:
X_set = windows_train
Y_set = train_Y

In [17]:
function feval(w_new)
   
    bsize= batch
    local idx = torch.randperm(X_set:size(1)):sub(1,bsize)

    local x = torch.Tensor(bsize, X_set:size(2))

    
    --y = y:reshape(Y_train:size(1), 1)
    
    local y_ = torch.Tensor(bsize, 1)

    for i=1,bsize do
        x[i] = X_set[idx[i]]
        y_[i] = Y_set[idx[i]]
    end
    
    y_ = y_:squeeze()
    local inputs = x
    local targets = y_
    -- reset gradients (gradients are always accumulated, to accommodate
    -- batch methods)
    dl_dw:zero()
    lookup1.weight:renorm(2, 1, 5)
    -- evaluate the loss function and its derivative with respect to x, given a mini batch
    local prediction = h:forward(inputs)
    local loss_w = mse:forward(prediction, targets)
    h:backward(inputs, mse:backward(prediction, targets))
    
    return loss_w, dl_dw
            
end

batch = 10

-- cycle on data
for i = 1,100 do
    -- train a mini_batch of batchSize in parallel
    _, fs = optim.adam(feval,w)

    if i % 10 == 0 then
        print('loss for iteration ' .. i  .. ' is ' .. fs[1] )
        -- print(sgd_params)
    end
end

loss for iteration 10 is 0.67658986910643	


loss for iteration 20 is 0.13615051635311	


loss for iteration 30 is 0.73139965654037	


loss for iteration 40 is 0.51190467823095	


loss for iteration 50 is 0.30367351892999	


loss for iteration 60 is 0.24954600970622	


loss for iteration 70 is 0.43246500995746	


loss for iteration 80 is 0.3343019728777	


loss for iteration 90 is 0.53911550107403	


loss for iteration 100 is 0.15713514378414	


In [10]:
function perplexity(input_set, output_set)
    sum = 0
    distributions = torch.Tensor(input_set:size(1),2)
    for i = 1,input_set:size(1) do
        a = (h:forward(input_set[i]))
        
        distribution = (nn.LogSoftMax():forward(a))
        distributions[i] = torch.exp(distribution)
        answer = distribution[output_set[i]]
        sum = sum + answer
        
    end
    return torch.exp(-sum/input_set:size(1)), distributions
end


In [19]:
perp, dist = perplexity(windows_valid:sub(1,windows_valid:size(1)), valid_reduced_Y)
print(perp)

In [40]:
nn

0.14645153348794	


In [ ]:
distributions:size()

In [ ]:
function write2file(scores, fname)
    f = io.open(fname, "w")
    f:write("ID,Class1,CLass2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,CLass13,Class14,Class15,Class16,Class17,Class18,Class19,Class20,Class21,Class22,Class23,CLass24,Class25,Class26,Class27,Class28,Class29,Class30,Class31,Class32,Class33,Class34,CLass35,Class36,Class37,Class38,Class39,Class40,Class41,Class42,Class43,Class44,Class45,CLass46,Class47,Class48,Class49,Class50\n")
    for i=1,scores:size(1) do
        s = tostring(i)
        for j=1, scores:size(2) do
            s = s .. "," .. tostring(scores[i][j])
        end
        f:write(s .. "\n")
    end
    f:close()
end

In [ ]:
write2file(distributions, 'predictions.txt')